# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
from pathlib import Path # for generating file paths
import os # for creating folders

from functools import reduce

import re

import math
from tqdm import tqdm
import numpy as np

from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
from shapely.geometry import Point

import requests
import sqlalchemy as db

import warnings

from typing import List, Tuple, Union, Any, Dict

warnings.filterwarnings('ignore')

In [2]:
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_DIRECTORY = 'taxi_data'  # directory to save taxi data
TAXI_PARQUET = 'yellow_tripdata_{:04d}-{:02d}.parquet'  # taxi data filename

SHAPE_DIRECTORY = 'taxi_zones'  # directory to save shape file
SHAPE_FILE = Path(SHAPE_DIRECTORY, 'taxi_zones.shp')  # shape file path
UBER_DIRECTORY = 'uber_data'  # directory to save uber data
UBER_CSV = Path(UBER_DIRECTORY, 'uber_rides_sample.csv')  # uber file path

WEATHER_DIRECTORY = 'weather_data'  # directory to save weather data
WEATHER_URL = 'https://www.ncei.noaa.gov/data/local-climatological-data/access/{}/72505394728.csv'  # url to download weather data
CLEAN_DIRECTORY = 'cleaned_data'

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
AVG_EARTH_RADIUS_KM = 6371.0088  # average earth radius (km), used in calculating distance acorrding to latitude and longitude

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
# create folders
os.makedirs(TAXI_DIRECTORY, exist_ok=True)
os.makedirs(SHAPE_DIRECTORY, exist_ok=True)
os.makedirs(UBER_DIRECTORY, exist_ok=True)
os.makedirs(WEATHER_DIRECTORY, exist_ok=True)
os.makedirs(CLEAN_DIRECTORY, exist_ok=True)

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [x] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance


In [4]:
def calculate_distance(from_coord: Tuple[float], to_coord: Tuple[float]) -> float:
    '''calculate and return the distance

    Keyword arguments:
    from_coord -- the coordinates we start with
    to_coord -- the coordinates we end with
    
    '''
    # starting latitude
    lat1, lon1 = from_coord
    # ending latitude
    lat2, lon2 = to_coord
    
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    # calculate the distance of two latitudes and longitudes
    lat = lat2 - lat1
    lon = lon2 - lon1
    
    # calculate the distance
    d = math.sin(lat * 0.5) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(lon * 0.5) ** 2
    
    return 2 * AVG_EARTH_RADIUS_KM * math.asin(math.sqrt(d))  


In [5]:
def add_distance_column(data: pd.DataFrame, lat1_name: str, lat2_name: str, lon1_name: str, lon2_name: str) -> pd.DataFrame:
    '''add a distance column to the data and return the data

    Keyword arguments:
    data -- the dataframe we are targeted 
    lat1_name, lat2_name, lon1_name, lon2_name -- the columns we want to add
    
    '''
    data['distance'] = data[[lat1_name, lat2_name, lon1_name, lon2_name]].apply(
        lambda x: calculate_distance((x[lat1_name], x[lon1_name]), (x[lat2_name], x[lon2_name])), axis=1
    )
    
    return data


### Processing Shp Data

shp file is downloaded via https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page, unzipped mannually, and saved into ./taxi_zones

In [6]:
def load_and_clean_shp_data() -> pd.DataFrame:
    '''load, clean, and return cleaned shp file data'''
    shp = gpd.read_file(SHAPE_FILE)
    
    # convert coordinates of geometric zones to real latitude/longitude coordinates
    shp['lat'] = shp.to_crs(4326).centroid.y
    shp['lon'] = shp.to_crs(4326).centroid.x
    
    # select and rename columns
    shp = shp[['LocationID', 'lat', 'lon']]
    shp.columns = ['location', 'lat', 'lon']
    
    return shp

### Processing Uber Data

In [7]:
def load_and_clean_uber_data(csv_file: Union[str, Path]) -> pd.DataFrame:
    '''load and clean the uber data

    Keyword argument:
    csv_file -- file we want to load and clean
    
    '''
    # load uber data
    data = pd.read_csv(csv_file)
    
    # select columns
    data = data[['pickup_datetime', 'fare_amount', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']]
    
    # removing invalid data points
    data = data[(data['fare_amount'] >= 1) & (data['passenger_count'] >= 1)]
    
    # normalize the dtype for time column
    utc = data['pickup_datetime'].iloc[0][-3:]
    data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'].str.slice(0, -4), utc=utc)
    
    # select rides whose pickup location and dropoff location lie in the specified box
    data = data[(data['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) & \
                (data['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0]) & \
                (data['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) & \
                (data['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1]) & \
                (data['dropoff_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) & \
                (data['dropoff_latitude'] <= NEW_YORK_BOX_COORDS[1][0]) & \
                (data['dropoff_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) & \
                (data['dropoff_longitude'] <= NEW_YORK_BOX_COORDS[1][1])]
    
    # drop duplicates
    data = data.drop_duplicates()
    
    data = data[['pickup_datetime', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'fare_amount', 'passenger_count']]
    
    return data

In [8]:
def get_uber_data():
    '''return the full uber data with distance columns added'''
    
    data = load_and_clean_uber_data(UBER_CSV)
    data = add_distance_column(data, lat1_name='pickup_latitude', lon1_name='pickup_longitude', lat2_name='dropoff_latitude', lon2_name='dropoff_longitude')
    
    # regroup to monthly frequency
    month_freq = data.resample('M', on='pickup_datetime')['pickup_datetime'].count().to_frame('frequency')
    
    return data, month_freq

# load and clean uber data and count number of rides in monthly uber data
uber_data, uber_month_freq = get_uber_data()
uber_data.to_parquet(Path(CLEAN_DIRECTORY, 'uber.parquet'))

In [9]:
uber_data.head()

,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,fare_amount,passenger_count,distance
0,2015-05-07 19:52:06+00:00,40.738354,-73.999817,40.723217,-73.999512,7.5,1,1.683325
1,2009-07-17 20:04:56+00:00,40.728225,-73.994355,40.750325,-73.994710,7.7,1,2.457593
2,2009-08-24 21:45:00+00:00,40.740770,-74.005043,40.772647,-73.962565,12.9,1,5.036384
3,2009-06-26 08:22:21+00:00,40.790844,-73.976124,40.803349,-73.965316,5.3,3,1.661686
4,2014-08-28 17:47:00+00:00,40.744085,-73.925023,40.761247,-73.973082,16.0,5,4.475456


In [10]:
uber_month_freq.head()

,frequency
pickup_datetime,
2009-01-31 00:00:00+00:00,2503
2009-02-28 00:00:00+00:00,2261
2009-03-31 00:00:00+00:00,2665
2009-04-30 00:00:00+00:00,2531
2009-05-31 00:00:00+00:00,2619


### Downloading Taxi Data



In [11]:
def find_taxi_parquet_urls():
    '''return all links about yellow taxi trip records for all years available'''
    # fetch the url contents
    # the main page
    page = requests.get(TAXI_URL)  
    page = BeautifulSoup(page.content)
    
    # find all years
    years = page.find_all(class_='faq-answers')  
    # find all links about yellow taxi in each year
    links = [year.find_all(title='Yellow Taxi Trip Records') for year in years]
    # with format List[List[str]], get the links about yellow taxi for all years
    links = [[j.get_attribute_list('href')[0] for j in i] for i in links] 
    
    links = reduce(lambda x, y: x + y, links) 
    
    return links

In [12]:
def get_month_taxi_data(url: str) -> None:
    '''download the taxi data into the file named taxi_data with url provided'''
    # e.g. './taxi_data/yellow_tripdata_2022-01.parquet'
    path = Path('taxi_data', url.split('/')[-1])  
    
    if path.exists():
        # already downloaded, skip this file
        return
    # not downloaded yet, download now
    data = requests.get(url)
    
    # save raw data
    with open(path, 'wb') as f:
        f.write(data.content)
    f.close()
    

In [13]:
def get_taxi_data() -> None:
    '''return raw taxi data'''
    # get all taxi data's urls
    all_taxi_urls = find_taxi_parquet_urls()
    
    for parquet_url in tqdm(all_taxi_urls, ncols=120, desc='downloading taxi data'):
        data = get_month_taxi_data(parquet_url)
        
# download taxi data for each month
taxi_data = get_taxi_data()


downloading taxi data: 100%|████████████████████████████████████████████████████████| 165/165 [00:00<00:00, 1492.04it/s]


### Processing Taxi Data

The columns names of taxi data for different year varies:

During year 2009, the column names are
- ['vendor_name', 'Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime', 'Passenger_Count', 'Trip_Distance', 'Start_Lon', 'Start_Lat', 'Rate_Code', 'store_and_forward', 'End_Lon', 'End_Lat', 'Payment_Type', 'Fare_Amt', 'surcharge', 'mta_tax', 'Tip_Amt', 'Tolls_Amt', 'Total_Amt']

During year 2010, the column names are
- ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']

From year 2011 to now, the column names are
- ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']

And for year 2009 and 2010, the latitidue and longitude of pickup and dropoff location are directly given in the dataset, whereas in year 2011 and after, only location IDs are given. 

In [14]:
def load_and_clean_taxi_data(parquet_file: Union[str, Path], shp: pd.DataFrame, year: int) -> pd.DataFrame:
    '''load, clean, and return the cleaned taxi data

    Keyword arguments:
    parquet_file -- file we want to load and clean
    shp -- the file our coordinates based on
    year -- data with specific year we wish to clean upon
    
    '''
    
    # load data
    data = pd.read_parquet(parquet_file)
    # select and rename columns
    if year == 2009:
        data = data[['Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime', 'Start_Lat', 'Start_Lon', 'End_Lat', 'End_Lon', 'Fare_Amt', 'Tip_Amt', 'Passenger_Count']]
        data.columns = ['pickup_datetime', 'dropoff_datetime', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'fare_amount', 'tip_amount', 'passenger_count']
    elif year == 2010:
        data = data[['pickup_datetime', 'dropoff_datetime', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'fare_amount', 'tip_amount', 'passenger_count']]
    else:
        data = data[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'fare_amount', 'tip_amount', 'passenger_count']]
        data.columns = ['pickup_datetime', 'dropoff_datetime', 'pickup_location', 'dropoff_location', 'fare_amount', 'tip_amount', 'passenger_count']
    # removing invalid data points
    data = data[(data['fare_amount'] >= 1) & (data['tip_amount'] >= 0) & (data['passenger_count'] >= 1)]
    # merge with shape file to get specific latitude/longitude coordinates
    if year >= 2011:
        data = pd.merge(data, shp, left_on='pickup_location', right_on='location')
        data = data.rename(columns={'lat': 'pickup_latitude', 'lon': 'pickup_longitude'})
        data = pd.merge(data, shp, left_on='dropoff_location', right_on='location')
        data = data.rename(columns={'lat': 'dropoff_latitude', 'lon': 'dropoff_longitude'})
    # select rides whose pickup location and dropoff location lie in the specified box
    data = data[(data['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) & \
                (data['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0]) & \
                (data['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) & \
                (data['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1]) & \
                (data['dropoff_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) & \
                (data['dropoff_latitude'] <= NEW_YORK_BOX_COORDS[1][0]) & \
                (data['dropoff_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) & \
                (data['dropoff_longitude'] <= NEW_YORK_BOX_COORDS[1][1])]
    # select columns
    data = data[['pickup_datetime', 'dropoff_datetime', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'fare_amount', 'tip_amount', 'passenger_count']]
    # normalize column dtypes
    data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
    data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])
    data['passenger_count'] = data['passenger_count'].astype(int)
    return data

In [15]:
def sample_taxi_data(data: pd.DataFrame, n: int, random_state: int = 0) -> pd.DataFrame:
    '''randomly sampling taxi data without replacement and return the sampled data

    Keyword arguments:
    data -- file we want to sample from
    n -- number of random sampling we wish to perform
    random state -- random state our random sampling based on (default = 0)
    
    '''
    
    # the number of samples shall not exceed the total samples in the dataset
    n = min(n, data.shape[0])  
    
    # perform random sampling
    data = data.sample(n, replace=False, random_state=random_state)
    
    return data

In [16]:
def get_month_taxi_data(parquet_file: Union[str, Path], shp: pd.DataFrame, n: int, year: int, random_state: int = 0) -> pd.DataFrame:
    '''return the monthly taxi data 

    Keyword arguments:
    parquet_file -- file we load in
    shp -- the shp file we based on
    n -- number of random sampling we wish to perform
    year -- specify specific year we want
    random state -- random state our random sampling based on (default = 0)
    
    '''
    
    if not os.path.exists(parquet_file):
        print(f'File {parquet_file} not existed, skip this file.')
        return pd.DataFrame()
    
    data = load_and_clean_taxi_data(parquet_file=parquet_file, shp=shp, year=year)
    data = sample_taxi_data(data=data, n=n, random_state=random_state)
    data = add_distance_column(data, lat1_name='pickup_latitude', lon1_name='pickup_longitude', 
                               lat2_name='dropoff_latitude', lon2_name='dropoff_longitude')
    return data


In [17]:
def get_taxi_data(sample_freq, random_state: int = 0) -> pd.DataFrame:
    '''return the final cleaned taxi data 

    Keyword arguments:
    sample_freq -- the sampled data with frequency specified
    random state -- random state our random sampling based on (default = 0)
    
    '''    
    taxi_data = []
    shp = load_and_clean_shp_data()
    
    for year in range(2009, 2023):
        for month in tqdm(range(1, 13), ncols=120, desc=f'getting taxi data of {year:04d}'):
            parquet_file = Path(TAXI_DIRECTORY, TAXI_PARQUET.format(year, month))
            
            try:
                n = sample_freq.loc[f'{year:04d}-{month:02d}'].squeeze()
           
            except KeyError:
                # if uber data does not contain this period of time, set n to 2000
                n = 2000
            
            data = get_month_taxi_data(parquet_file=parquet_file, shp=shp, n=n, year=year, random_state=random_state)
            taxi_data.append(data)
    
    taxi_data = pd.concat(taxi_data, axis=0, ignore_index=True)
    return taxi_data

In [18]:
taxi_data = get_taxi_data(uber_month_freq)

taxi_data.to_parquet(Path(CLEAN_DIRECTORY, 'taxi.parquet'))

getting taxi data of 2022: 100%|████████████████████████████████████████████████████████| 12/12 [00:33<00:00,  2.79s/it]


File taxi_data/yellow_tripdata_2022-10.parquet not existed, skip this file.
File taxi_data/yellow_tripdata_2022-11.parquet not existed, skip this file.
File taxi_data/yellow_tripdata_2022-12.parquet not existed, skip this file.


### Downloading Weather Data

In [19]:
def get_weather_data(start: int = 2005, end: int = 2022) -> None:
    '''download weather data to local file

    Keyword arguments:
    start -- start year (default = 2005)
    end -- end year (default = 2022)
    
    '''
    for year in tqdm(range(start, end + 1), ncols=120, desc='downloading weather data'):
        path = Path('weather_data', str(year) + '.csv')
        
        if path.exists():
            # already downloaded, skip this file
            continue
            
        # not downloaded yet, download now
        data = pd.read_csv(WEATHER_URL.format(year))
        
        data.to_csv(path)

In [20]:
get_weather_data()

downloading weather data: 100%|███████████████████████████████████████████████████████| 18/18 [00:00<00:00, 2270.74it/s]


### Processing Weather Data


the weather data contains the following:

* HourlyPrecipitation: contains numbers, strings of numbers (such as '0.12s'), NaNs, and 'T'. Need to convert 'T' to 0.0, use regex to extract the numbers, and treat non-number-strings as NaNs

* HourlyWindSpeep: contains numbers, NaNs

* HourlyDryBuldTemperature: contains numbers, strings of numbers (such as '52.0'), NaNs, and strings of numbers and other chars (such as '81s'). Need to first use regex to extract the numbers, and treat non-number-strings as NaNs

* HourlyVisibility: same as HourlyDryBuldTemperature

* NaNs policy: sort ascendingly on time, and fill all NaNs with precedent observations (method='ffill')

In [21]:
def normalize_weather_data(x: Union[str, int, float]) -> float:
    '''return normalized data; if data does not exist, return NaN

    Keyword arguments:
    x -- the value we wish to normalize
    
    '''
    # conver to string
    x = str(x)
    
    # find all the integers and floats
    x = re.findall('[\d|\.]+', x) 
    
    if len(x) >= 1:
        return float(x[0])
    else:
        # find no numbers
        return np.nan

In [22]:
def clean_month_weather_data_hourly(data: pd.DataFrame) -> pd.DataFrame:
    '''return the cleaned hourly weather data

    Keyword argument:
    data -- the dataframe we wish to clean
    
    '''
    # select necessary columns
    data = data[['DATE', 'LATITUDE', 'LONGITUDE', 
                 'HourlyDryBulbTemperature', 'HourlyPrecipitation', 
                 'HourlyVisibility', 'HourlyWindSpeed']]
    
    # drop entries which are all NaNs
    # since 'DATE', 'LATITUDE', 'LONGITUDE' has no NaNs, we don't need to consider them when drop NaNs
    data = data.set_index(['DATE', 'LATITUDE', 'LONGITUDE']).dropna(how='all').reset_index()
    
    # convert time to datetime
    data['DATE'] = pd.to_datetime(data['DATE'])
    
    data['HourlyPrecipitation'] = data['HourlyPrecipitation'].apply(lambda x: 0. if x == 'T' else x)
    data['HourlyPrecipitation'] = data['HourlyPrecipitation'].apply(normalize_weather_data)
    data['HourlyWindSpeed'] = data['HourlyWindSpeed'].apply(float)
    data['HourlyDryBulbTemperature'] = data['HourlyDryBulbTemperature'].apply(normalize_weather_data)
    data['HourlyVisibility'] = data['HourlyVisibility'].apply(normalize_weather_data)
    
    data.columns = ['datetime', 'latitude', 'longitude', 'temperature', 'precipitation', 'visibility', 'wind_speed']
    
    data = data.sort_values('datetime')
    data = data.fillna(method='ffill')
    
    return data


In [23]:
def clean_weather_data_daily(data: pd.DataFrame) -> pd.DataFrame:
    '''return the cleaned daily weather data

    Keyword argument:
    data -- the (hourly) dataframe we wish to group up and clean
    
    '''
    # the method applied while grouping by day for each column
    group_method = {
        'temperature': 'mean',
        'precipitation': 'sum',
        'visibility': 'mean',
        'wind_speed': 'mean',
    }
    
    daily_data = data.resample('D', on='datetime').agg(group_method)
    
    daily_data.insert(0, 'latitude', data['latitude'].iloc[0])
    daily_data.insert(1, 'longitude', data['longitude'].iloc[0])
    
    daily_data = daily_data.reset_index()
    
    return daily_data

In [24]:
def load_and_clean_weather_data() -> Tuple[pd.DataFrame, pd.DataFrame]:
    '''return a tuple cleaned hourly and daily weather data'''
    
    hourly_dataframes = []
    weather_csv_files = os.listdir('weather_data')
    
    for csv_file in tqdm(weather_csv_files, ncols=120, desc='load and clean weather data'):
        data = pd.read_csv(Path(WEATHER_DIRECTORY, csv_file), index_col=0)
        hourly_dataframe = clean_month_weather_data_hourly(data)
        hourly_dataframes.append(hourly_dataframe)
        
    # create hourly data from every month
    hourly_data = pd.concat(hourly_dataframes)
    
    daily_data = clean_weather_data_daily(hourly_data)
    
    return hourly_data, daily_data

In [25]:
hourly_data, daily_data = load_and_clean_weather_data()

hourly_data.to_parquet(Path(CLEAN_DIRECTORY, 'hourly_weather.parquet'))
daily_data.to_parquet(Path(CLEAN_DIRECTORY, 'daily_weather.parquet'))


load and clean weather data: 100%|██████████████████████████████████████████████████████| 18/18 [00:05<00:00,  3.50it/s]


In [26]:
hourly_data.head()

,datetime,latitude,longitude,temperature,precipitation,visibility,wind_speed
0,2008-01-01 00:51:00,40.77898,-73.96925,37.0,NaN,10.0,0.0
1,2008-01-01 01:51:00,40.77898,-73.96925,37.0,NaN,9.0,6.0
2,2008-01-01 02:51:00,40.77898,-73.96925,39.0,NaN,10.0,6.0
3,2008-01-01 03:51:00,40.77898,-73.96925,39.0,NaN,10.0,5.0
4,2008-01-01 04:51:00,40.77898,-73.96925,39.0,NaN,9.0,5.0


In [27]:
daily_data.head()

,datetime,latitude,longitude,temperature,precipitation,visibility,wind_speed
0,2005-01-01,40.77898,-73.96925,51.083333,0.00,9.875000,8.666667
1,2005-01-02,40.77898,-73.96925,42.250000,0.00,10.000000,7.916667
2,2005-01-03,40.77898,-73.96925,51.833333,0.43,7.260417,5.666667
3,2005-01-04,40.77898,-73.96925,47.636364,0.25,8.795455,4.363636
4,2005-01-05,40.77898,-73.96925,39.125000,0.44,7.416667,6.541667


## Part 2: Storing Cleaned Data


In [28]:
engine = db.create_engine(DATABASE_URL)

In [29]:
# schema for hourly weather data
HOURLY_WEATHER_SCHEMA = """CREATE TABLE IF NOT EXISTS HOURLY_WEATHER(
    hourly_weather_uid INTEGER PRIMARY KEY AUTOINCREMENT,
    datetime DATETIME,
    latitude FLOAT,
    longitude FLOAT,
    temperature FLOAT,
    precipitation FLOAT,
    visibility FLOAT,
    wind_speed FLOAT
)"""
HOURLY_WEATHER_DTYPE = {
    'datetime': db.DateTime(),
    'latitude': db.Float(),
    'longitude': db.Float(),
    'temperature': db.Float(),
    'precipitation': db.Float(),
    'visibility': db.Float(),
    'wind_speed': db.Float(),
}

In [30]:
# schema for daily weather data

DAILY_WEATHER_SCHEMA = """CREATE TABLE IF NOT EXISTS DAILY_WEATHER(
    daily_weather_uid INTEGER PRIMARY KEY AUTOINCREMENT,
    datetime DATETIME,
    latitude FLOAT,
    longitude FLOAT,
    temperature FLOAT,
    precipitation FLOAT,
    visibility FLOAT,
    wind_speed FLOAT
)"""
DAILY_WEATHER_DTYPE = {
    'datetime': db.DateTime(),
    'latitude': db.Float(),
    'longitude': db.Float(),
    'temperature': db.Float(),
    'precipitation': db.Float(),
    'visibility': db.Float(),
    'wind_speed': db.Float(),
}

In [31]:
# schema for taxi data

TAXI_TRIPS_SCHEMA = """CREATE TABLE IF NOT EXISTS TAXI_TRIPS(
    taxi_trips_uid INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    pickup_latitude FLOAT,
    pickup_longitude FLOAT,
    dropoff_latitude FLOAT,
    dropoff_longitude FLOAT,
    fare_amount FLOAT,
    tip_amount FLOAT,
    passenger_count INTEGER,
    distance FLOAT
)"""
TAXI_TRIPS_DTYPE = {
    'pickup_datetime': db.DateTime(),
    'dropoff_datetime': db.DateTime(),
    'pickup_latitude': db.Float(),
    'pickup_longitude': db.Float(),
    'dropoff_latitude': db.Float(),
    'dropoff_longitude': db.Float(),
    'fare_amount': db.Float(),
    'tip_amount': db.Float(),
    'passenger_count': db.Integer(),
    'distance': db.Float(),
}

In [32]:
# schema for uber data

UBER_TRIPS_SCHEMA = """CREATE TABLE IF NOT EXISTS UBER_TRIPS(
    uber_trips_uid INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATETIME,
    pickup_latitude FLOAT,
    pickup_longitude FLOAT,
    dropoff_latitude FLOAT,
    dropoff_longitude FLOAT,
    fare_amount FLOAT,
    tip_amount FLOAT,
    distance FLOAT
)"""
UBER_TRIPS_DTYPE = {
    'pickup_datetime': db.DateTime(),
    'pickup_latitude': db.Float(),
    'pickup_longitude': db.Float(),
    'dropoff_latitude': db.Float(),
    'dropoff_longitude': db.Float(),
    'fare_amount': db.Float(),
    'tip_amount': db.Float(),
    'distance': db.Float(),
}

In [33]:
# create the required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [34]:
# create the tables with the schema files
with engine.connect() as connection:
    connection.execute(HOURLY_WEATHER_SCHEMA)
    connection.execute(DAILY_WEATHER_SCHEMA)
    connection.execute(TAXI_TRIPS_SCHEMA)
    connection.execute(UBER_TRIPS_SCHEMA)

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [46]:
def write_dataframes_to_table(table_to_df_dict: Dict[str, pd.DataFrame], table_to_dtype_dict: Dict[str, Dict[str, db.types.TypeEngine]]):
    '''add dataframes to the database
    
    Keyword arguments:
    table_to_df_dict -- the dictionary that stores the data
    table_to_dtype_dict -- the dictionary that stores the corresponding dtypes
    '''
    with engine.connect() as connection: 
        for name, data in table_to_df_dict.items():
            print(f'Saving data to {name} with entries {data.shape[0]} ...', end=' ')
            data.to_sql(name=name, con=connection, if_exists='replace', index=False, dtype=table_to_dtype_dict[name])
            print('suceeded.')

In [47]:
map_table_name_to_dataframe = {
    "TAXI_TRIPS": taxi_data,
    "UBER_TRIPS": uber_data,
    "HOURLY_WEATHER": hourly_data,
    "DAILY_WEATHER": daily_data,
}
map_table_name_to_dtype = {
    "TAXI_TRIPS": TAXI_TRIPS_DTYPE,
    "UBER_TRIPS": UBER_TRIPS_DTYPE,
    "HOURLY_WEATHER": HOURLY_WEATHER_DTYPE,
    "DAILY_WEATHER": DAILY_WEATHER_DTYPE,
}

In [48]:
write_dataframes_to_table(map_table_name_to_dataframe, map_table_name_to_dtype)

Saving data to TAXI_TRIPS with entries 368766 ... suceeded.
Saving data to UBER_TRIPS with entries 194766 ... suceeded.
Saving data to HOURLY_WEATHER with entries 195391 ... suceeded.
Saving data to DAILY_WEATHER with entries 6550 ... suceeded.


## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)